### A.2 Function to Create Silhouette Plot

In [1]:
# ==================================================================================

def silhouette_plot(X, cluster_labels, ax=None, tot_time=None):
    """
    Plot Silhouette graph based on cluster_labels obtained from the clustering algorithms
    Note: This function works only when the number of clusters in cluster_labels is > 1 because of
    computational problem with silhouett_score.
    
    Parameters:
    ----------
    
    X: an array shape: (n_samples, n_attributes)
        It is data matrix
        
    cluster_labels: an array shape: n_samples
        It is the labels assigned by a clustering algorithm
        
    ax: a tuple of two elements
        It represents axis objects 
        
    tot_time: a scalar
        It represents the time to execute a clustering alrorithm given the parameters and data 
        
        
    Return:
    ------
    
    None
    
    
    indices_negative_silhouette: a dictionary
    
        It contains the indices of data points in each cluster whose silhouette scores are negative.
    
    """
    
    # Acknowledgement: the codes in this function are taken from following website of sklearn
    # https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
    
    
    n_clusters = len(np.unique(cluster_labels))
    negative_silhouette_indices = {}
    # Single cluster creates an empty plot
    if n_clusters > 1:
        
        if ax == None:
#             fig, (ax1, ax2) = plt.subplots(1, 2)
            fig = plt.figure(figsize=(2*f_w,f_h))
            ax1 = fig.add_subplot(1, 2, 1)
            ax2 = fig.add_subplot(1, 2, 2)
#             fig.set_size_inches(2*f_w, 1*f_h)
               
        else:
            # when two axes are provided
            ax1 = ax[0]
            ax2 = ax[1]

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1
        ax1.set_xlim([-0.5, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        silhouette_avg = metrics.silhouette_score(X, cluster_labels)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = metrics.silhouette_samples(X, cluster_labels)        
        mask_neg_score = sample_silhouette_values < 0

        y_lower = 15
#         for i in range(n_clusters):
        for l,i in enumerate(np.unique(cluster_labels)):
            # store indices of negative silhouette score in each cluster
            if i != -1: # only for non-negative clusters
                mask_cluster = cluster_labels == i
                negative_silhouette_indices[i] =  mask_neg_score * mask_cluster
            
            
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            # grey color for noise points            
            if i == -1:
                color = 'gray'
            else:
                color = cm.nipy_spectral(float(i) / n_clusters)                     
            
            
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.9)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("Silhouette plot: " + str(np.round(silhouette_avg, 3)))
        ax1.set_xlabel("Silhouette coefficient")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="dashdot")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
        
       
        # 2nd Plot showing the actual clusters formed

        # find the cluster where negative silhouette score occured
        cluster_neg_silhouette = []
        for key, value in negative_silhouette_indices.items():
            if np.any(value):
                cluster_neg_silhouette.append(key)

#         print()
#         print("Cluster with negative Silhouette score")
#         print(cluster_neg_silhouette)

        colors = cm.nipy_spectral(cluster_labels / n_clusters)

        if np.any(np.unique(cluster_labels) == -1):
            mask = cluster_labels == -1
            ax2.scatter(X[:, 0][mask], X[:, 1][mask],
                    marker='*', 
                    s=size, 
                    alpha=aph,
                    c='grey', 
#                     edgecolor='k'
                )

            ax2.scatter(X[:, 0][~mask], X[:, 1][~mask],
                    marker='.', 
                    s=size, 
                    alpha=aph,
                    c=colors[~mask], 
#                     edgecolor='k'
                )
            # if there are negative sihouette points
            if len(cluster_neg_silhouette) > 0:
                for c in cluster_neg_silhouette:
                    ax2.scatter(X[:, 0][negative_silhouette_indices[c]], X[:, 1][negative_silhouette_indices[c]],
                                marker='x',
                                s=size+5, 
                                alpha=aph,
                                c=colors[negative_silhouette_indices[c]]
                                #                     edgecolor='k'
                               )


        else:
            ax2.scatter(X[:, 0], X[:, 1],
                        marker='.', 
                        s=size, 
                        alpha=aph,
                        c=colors, 
    #                     edgecolor='k'
                    )
            # if there are negative sihouette points
            if len(cluster_neg_silhouette) > 0:
                for c in cluster_neg_silhouette:
                    ax2.scatter(X[:, 0][negative_silhouette_indices[c]], X[:, 1][negative_silhouette_indices[c]],
                                marker='x',
                                s=size+5, 
                                alpha=aph,
                                c=colors[negative_silhouette_indices[c]]
                                #                     edgecolor='k'
                               )
        ax2.axis('equal')

        if tot_time == None:
            ax2.set_title("Clustered Outputs") 
        else:            
            ax2.set_title("Result: {:.4f} sec".format(tot_time))   
            
        plt.show()

    return None



def silhouette_plot_3d(X, cluster_labels, ax=None, tot_time=None):
    """
    Plot Silhouette graph based on cluster_labels obtained from the clustering algorithms
    Note: This function works only when the number of clusters in cluster_labels is > 1 because of
    computational problem with silhouett_score.
    
    Parameters:
    ----------
    
    X: an array shape: (n_samples, n_attributes)
        It is data matrix
        
    cluster_labels: an array shape: n_samples
        It is the labels assigned by a clustering algorithm
        
    ax: a tuple of two elements
        It represents axis objects 
        
    tot_time: a scalar
        It represents the time to execute a clustering alrorithm given the parameters and data 
        
        
    Return:
    ------
    
    None
    
    
    indices_negative_silhouette: a dictionary
    
        It contains the indices of data points in each cluster whose silhouette scores are negative.
    
    """
    
    # Acknowledgement: the codes in this function are taken from following website of sklearn
    # https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
    
    
    n_clusters = len(np.unique(cluster_labels))
    negative_silhouette_indices = {}

    if n_clusters > 1:
        
        if ax == None:

            fig = plt.figure(figsize=(2*f_w,f_h))
            ax1 = fig.add_subplot(1, 2, 1)
            ax2 = fig.add_subplot(1, 2, 2, projection='3d')

               
        else:
            # when two axes are provided
            ax1 = ax[0]
            ax2 = ax[1]
           
        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1
        ax1.set_xlim([-0.5, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        silhouette_avg = metrics.silhouette_score(X, cluster_labels)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = metrics.silhouette_samples(X, cluster_labels)        
        mask_neg_score = sample_silhouette_values < 0

        y_lower = 15
#         for i in range(n_clusters):
        for l,i in enumerate(np.unique(cluster_labels)):
            # store indices of negative silhouette score in each cluster
            if i != -1: # only for non-negative clusters
                mask_cluster = cluster_labels == i
                negative_silhouette_indices[i] =  mask_neg_score * mask_cluster
            
            
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            # grey color for noise points            
            if i == -1:
                color = 'gray'
            else:
                color = cm.nipy_spectral(float(i) / n_clusters)                     
            
            
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.9)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("Silhouette plot: " + str(np.round(silhouette_avg, 3)))
        ax1.set_xlabel("Silhouette coefficient")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="dashdot")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
        
        
        # ---------------------------------------------------------
        
        # 2nd Plot showing the actual clusters formed

        # find the cluster where negative silhouette score occured
        cluster_neg_silhouette = []
        for key, value in negative_silhouette_indices.items():
            if np.any(value):
                cluster_neg_silhouette.append(key)

#         print()
#         print("Cluster with negative Silhouette score")
#         print(cluster_neg_silhouette)

        colors = cm.nipy_spectral(cluster_labels / n_clusters)

        if np.any(np.unique(cluster_labels) == -1):
            mask = cluster_labels == -1
            ax2.scatter(X[:, 0][mask], X[:, 1][mask], X[:, 2][mask],
                    marker='*', 
                    s=size, 
                    alpha=aph,
                    c='grey', 
#                     edgecolor='k'
                )

            ax2.scatter(X[:, 0][~mask], X[:, 1][~mask], X[:, 2][~mask],
                    marker='.', 
                    s=size, 
                    alpha=aph,
                    c=colors[~mask], 
#                     edgecolor='k'
                )
            # if there are negative sihouette points
            if len(cluster_neg_silhouette) > 0:
                for c in cluster_neg_silhouette:
                    ax2.scatter(X[:, 0][negative_silhouette_indices[c]], X[:, 1][negative_silhouette_indices[c]],
                                X[:, 2][negative_silhouette_indices[c]],
                                marker='x',
                                s=size+5, 
                                alpha=aph,
                                c=colors[negative_silhouette_indices[c]]
                                #                     edgecolor='k'
                               )


        else:
            ax2.scatter(X[:, 0], X[:, 1],X[:, 2],
                        marker='.', 
                        s=size, 
                        alpha=aph,
                        c=colors, 
    #                     edgecolor='k'
                    )
            # if there are negative sihouette points
            if len(cluster_neg_silhouette) > 0:
                for c in cluster_neg_silhouette:
                    ax2.scatter(X[:, 0][negative_silhouette_indices[c]], X[:, 1][negative_silhouette_indices[c]],
                                X[:, 2][negative_silhouette_indices[c]],
                                marker='x',
                                s=size+5, 
                                alpha=aph,
                                c=colors[negative_silhouette_indices[c]]
                                #                     edgecolor='k'
                               )
#         ax2.axis('equal')

        if tot_time == None:
            ax2.set_title("Clustered Outputs") 
        else:            
            ax2.set_title("Result: {:.4f} sec".format(tot_time))   
        ax2.set_xlabel("x")
        ax2.set_ylabel("y")
        ax2.set_zlabel("z")
        # rotate 3d figure
        ax2.view_init(elev=20, azim=-50) # angles in degrees
        
        plt.show()
        
    return None


def silhouette_plot_only(X, cluster_labels, ax=None, name=" ", show_score=False):
    """
    Plot Silhouette graph based on cluster_labels obtained from the clustering algorithms
    Note: This function works only when the number of clusters in cluster_labels is > 1 because of
    computational problem with silhouett_score.
    
    Parameters:
    ----------
    
    X: an array shape: (n_samples, n_attributes)
        It is data matrix
        
    cluster_labels: an array shape: n_samples
        It is the labels assigned by a clustering algorithm
        
    ax: a tuple of two elements
        It represents axis objects 
        
    name: a string
        It represents the name of the algorithm to use for clustering
        
    show_score: a boolean
        If true then value of silhouette score is display in the title
        
        
    Return:
    ------
    
    avg_score: a double
        It represents the average silhouette score
    
    
#    indices_negative_silhouette: a dictionary
    
#        It contains the indices of data points in each cluster whose silhouette scores are negative.
    
    """
    
    # Acknowledgement: the codes in this function are taken from following website of sklearn
    # https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
    
    
    n_clusters = len(np.unique(cluster_labels))
    negative_silhouette_indices = {}
    
    silhouette_avg = 0

    if n_clusters > 1:
        
        if ax == None:
            fig = plt.figure(figsize=(f_w,f_h))
            ax1 = fig.add_subplot(1, 1, 1)
        else:
            # when axis is provided
            ax1 = ax

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1
        ax1.set_xlim([-0.5, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        silhouette_avg = metrics.silhouette_score(X, cluster_labels)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = metrics.silhouette_samples(X, cluster_labels)        
        mask_neg_score = sample_silhouette_values < 0

        y_lower = 15
#         for i in range(n_clusters):
        for l,i in enumerate(np.unique(cluster_labels)):
            # store indices of negative silhouette score in each cluster
            if i != -1: # only for non-negative clusters
                mask_cluster = cluster_labels == i
                negative_silhouette_indices[i] =  mask_neg_score * mask_cluster
            
            
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            # grey color for noise points            
            if i == -1:
                color = 'gray'
            else:
                color = cm.nipy_spectral(float(i) / n_clusters)                     
            
            
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.9)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

#         ax1.set_title(name+': '+ str(np.round(silhouette_avg, 3)))
        if show_score:
            ax1.set_title(str(np.round(silhouette_avg, 3)))
        else:            
            ax1.set_title(name)
            
        ax1.set_xlabel("Coefficients")
        ax1.set_ylabel("Cluster Labels")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="dashdot")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
    # return 0 value when there is only one cluster    
    return silhouette_avg


def get_and_plot_silhouette_scores(X, algo_predicted_label, algo_name):
    """
    
    """
    column = 4 # numbe of figures to be displayed in columns
    row = 2
    # get the current figure
    fig = plt.figure(figsize=(f_w*column, f_h*row))
#     fig.subplots_adjust(left=.1, right=0.85, bottom=0.1, top=0.9, wspace=0.02, hspace=0.2);
    plt.subplots_adjust(top=0.95, bottom=0.08, left=0.10, right=0.95, hspace=0.4,
                    wspace=0.35)
    # get the axes of the subplots
    ax = fig.subplots(row, column, sharex=True, sharey=True);
    fig_count = 0
    avg_score = []
    for r in range(row):
        for c in range(column):
#                 if fig_count < n_fig:
            temp = silhouette_plot_only(X, algo_predicted_label[fig_count], ax = ax[r, c], name=algo_name[fig_count])
            avg_score.append(temp)           
            fig_count += 1   
    
    
    return avg_score

